In [2]:
import Pkg;
Pkg.add("PyPlot")
# Pkg.add("CSV"); 
# Pkg.add("DataFrames");

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [3]:
using CSV
using DataFrames
using LinearAlgebra
using Plots

In [4]:
function get_actual_values_from_cumulatives(cumulative_vector)
    shifted_values = zeros(Float64, size(cumulative_vector)[1]+1)
    for (index, value) in enumerate(cumulative_vector)
       shifted_values[index+1] = value  
    end

    final_values = zeros(Float64, size(cumulative_vector)[1])
    for (index, value) in enumerate(cumulative_vector)
       final_values[index] = -shifted_values[index]+cumulative_vector[index]  
    end
    return final_values
end

get_actual_values_from_cumulatives (generic function with 1 method)

Read data. Data downloaded from https://github.com/pcm-dpc/COVID-19/blob/master/dati-andamento-covid19-italia.md

In [5]:
data_path = "../DDO_2/data/dpc-covid19-ita-andamento-nazionale.csv"

data = CSV.read(data_path, DataFrame)
# totale_positivi,dimessi_guariti,deceduti 
# (Total amount of current positive cases (Hospitalised patients + Home confinement))
# recovered
# deceased

infected = data[!, :"totale_positivi"]
recovered = data[!, :"dimessi_guariti"]
deceased = data[!, :"deceduti"]

628-element Vector{Int64}:
      7
     10
     12
     17
     21
     29
     34
     52
     79
    107
    148
    197
    233
      ⋮
 132120
 132161
 132224
 132283
 132334
 132365
 132385
 132423
 132491
 132551
 132618
 132686

Algorithm:

Equations:
    I(t) = αĨ(t)

    R(t) = αR̃(t)

    S(t) = αS̃(t)

where I(t), R(t) and S(t) are the actual number of infected, recovered and susceptible population at time t and α≥1;
   
    S(t₀) = ωP

where ωϵ[0,1] and P is the total population.

S̃(t + 1) = S̃(t) - β * (S̃(t) * Ĩ(t)) ÷ (S̃(t) + Ĩ(t))
Ĩ(t + 1) = Ĩ(t) + β * (S̃(t) * Ĩ(t)) ÷ (S̃(t) + Ĩ(t)) - γ * Ĩ(t) - ν * Ĩ(t)
R̃(t + 1) = R̃(t) + γ * Ĩ(t)
D(t + 1) = D(t) + α * ν * Ĩ(t)

where
    I(t) = αĨ(t)

    R(t) = αR̃(t)

    S(t) = αS̃(t)

where I(t), R(t) and S(t) are the actual number of infected, recovered and susceptible population at time t and α≥1;
   
    S(t₀) = ωP

where ωϵ[0,1] and P is the total population.

    S̃(t) = (ω ÷ α) * P - Ĩ(t) - R̃(t) - D̃(t)

    Δ(t) = [Ĩ(t + 1)-Ĩ(t); R̃(t + 1)-R̃(t); D̃(t + 1)-D̃(t)]
    Δ(t) = Φ_ωα(t) = [S̃(t)-β*(S̃(t)*Ĩ(t))÷(S̃(t)+Ĩ(t)) -Ĩ(t) -Ĩ(t)÷α; 0 Ĩ(t) 0; 0 0 Ĩ(t)]
    
    Δ̄ = [ρ^(Θ -t₀)Δ(t₀); ρ^(Θ -t₀-1)Δ(t₀+1); ρ^(Θ -t₀-2)Δ(t₀+2)...Δ(Θ)]
    Φ̄_ωα = [ρ^(Θ -t₀)Φ_ωα(t₀); ρ^(Θ -t₀-1)Φ_ωα(t₀+1); ρ^(Θ -t₀-2)Φ_ωα(t₀+2)...Φ_ωα(Θ)]

where ρϵ(0,1)


Steps:
1. Input: data Ĩ(t), R̃(t), and D(t), maximum value ᾱ of α, weighting parameter ρ, and total population P

outputs -> ω⋆, α⋆, β⋆, γ⋆, ν̄⋆


In [38]:
#we have Ĩ(t) and R̃(t) and D(t) in infected, recovered and deceased dataframes.
P = 332902734 #https://www.census.gov/popclock/
ᾱ = 101       # ᾱ*Infected + ᾱ*Recovered + ᾱ*Deceased < ω*Population
ρ = 0.9
ω = 0.05 
α_0 = 1
α_increment = 20
ω_increment = 0.05

num_α = Int64((ᾱ-α_0)/α_increment)+1
num_ω = Int64((1/ω_increment))

20

In [7]:
function get_susceptible(ω, α, P, Ĩ, R̃, D)
    population = zeros(Float64, size(Ĩ)[1])
    fill!(population, ω*P/α)
    return population-infected-recovered-deceased
end

get_susceptible (generic function with 1 method)

In [8]:
function invalid(vector_values)
    for (index, value) in enumerate(vector_values)
        if value < 0
            return true
        end
    end
    return false
end

invalid (generic function with 1 method)

In [9]:
function computeΔ̄(Ĩ, R̃, D, ρ)
    Θ = size(Ĩ)[1]
    
    ρmatrix = zeros(Float64, Θ)
    for i in 1:Θ
        power = Θ-i
        ρmatrix[i] = ρ^power
    end
    
    Δ̄ = zeros(Float64, 3Θ)
    for i in 1:Θ-1
        Δ̄[3(i-1)+1] = ρmatrix[i]*(Ĩ[i+1]-Ĩ[i])
        Δ̄[3(i-1)+2] = ρmatrix[i]*(R̃[i+1]-R̃[i])
        Δ̄[3(i-1)+3] = ρmatrix[i]*(D[i+1]-D[i])
    end
    
    Δ̄[3(Θ-1)+1] = ρmatrix[Θ]*(Ĩ[Θ])
    Δ̄[3(Θ-1)+2] = ρmatrix[Θ]*(R̃[Θ])
    Δ̄[3(Θ-1)+3] = ρmatrix[Θ]*(D[Θ])

    return Δ̄
end

computeΔ̄ (generic function with 1 method)

In [10]:
function computeΦ(ω, α, S̃, Ĩ)
    Θ = size(Ĩ)[1]
    
    ρmatrix = zeros(Float64, Θ)
    for i in 1:Θ
        power = Θ-i
        ρmatrix[i] = ρ^power
    end
    
    Φ = zeros(Float64, 3*Θ, 3)
    i=1
    while i<Θ
        Φ[i] = ρmatrix[i]*(S̃[i]*Ĩ[i])/ (S̃[i]+Ĩ[i])
        Φ[3*Θ+i] = -1*ρmatrix[i]*Ĩ[i]
        Φ[6*Θ+i] = -1*ρmatrix[i]*Ĩ[i]/α
        Φ[i+1] = 0
        Φ[3*Θ+i+1] = ρmatrix[i]*Ĩ[i]
        Φ[6*Θ+i+1] = 0
        Φ[i+2] = 0
        Φ[3*Θ+i+2] = 0
        Φ[6*Θ+i+2] = ρmatrix[i]*Ĩ[i]
        i+=3
    end
    return Φ
    
end

# S̃ = get_susceptible(0.5, 2, P, infected, recovered, deceased)
# computeΦ(0.5, 2, S̃, infected)


computeΦ (generic function with 1 method)

In [11]:
function computeParameters(Φ̄, Δ̄)
    @time pseudo_inv = pinv(Φ̄)
    params = pseudo_inv * Δ̄
    return params
end

computeParameters (generic function with 1 method)

In [12]:
function error_(Δ̄, Φ̄, params)
    param_matrix = zeros(Float64, 3, 1)
    param_matrix[1] = params[1]
    param_matrix[2] = params[2]
    param_matrix[3] = params[3]
    
    diff = Δ̄-(Φ̄*param_matrix)
    return norm(diff)^2
end

# M = [1.5 1.3; 1.2 1.9]
# N = [2,4]
# params = computeParameters(M, N)
# error_(N, M, params)
# # print(params)


error_ (generic function with 1 method)

In [39]:
function sir_model(Ĩ, R̃, D, P, ᾱ, ρ)
    e = Inf
    Δ̄ = computeΔ̄(Ĩ, R̃, D, ρ)
    ω_ = Inf
    α_ = Inf
    β_ = Inf
    γ_ = Inf
    ν̃_ = Inf
    
    ω = 0.05 
    α_0 = 1
    α_increment = 20
    ω_increment = 0.05
    
    num_α = Int64((ᾱ-α_0)/α_increment)+1
    num_ω = Int64((1/ω_increment))
    num_solve_points = (num_α*num_ω)
    loss_vals = zeros(Float64, num_ω, num_α)
    
    ω_store = zeros(Float64, num_ω, num_α)
    α_store = zeros(Float64, num_ω, num_α)
    β_store = zeros(Float64, num_ω, num_α)
    γ_store = zeros(Float64, num_ω, num_α)
    ν̃_store = zeros(Float64, num_ω, num_α)
    
    i = j = 1
    while ω ≤ 1
        α = α_0
        j=1
        while α ≤ ᾱ
            S̃ = get_susceptible(ω, α, P, Ĩ, R̃, D)
            if invalid(S̃)
                ω_store[i,j] = ω
                α_store[i,j] = α
                β_store[i,j] = 0
                γ_store[i,j] = 0
                ν̃_store[i,j] = 0
                loss_vals[i,j] = 0
                α+=α_increment
                j+=1
                continue
            end
            
            Φ̄ = computeΦ(ω, α, S̃, Ĩ)
            params = computeParameters(Φ̄, Δ̄)
            β = params[1]
            γ = params[2]
            ν̃ = params[3]
            if(β<0 || γ<0 || ν̃<0)
                ω_store[i,j] = ω
                α_store[i,j] = α
                β_store[i,j] = 0
                γ_store[i,j] = 0
                ν̃_store[i,j] = 0
                loss_vals[i,j] = 0
                α+=α_increment
                j+=1
                continue
            end

            error_value = error_(Δ̄, Φ̄, params)
            if error_value < e
                e = error_value
                ω_ = ω
                α_ = α
                β_ = β
                γ_ = γ
                ν̃_ = ν̃
            end

            ω_store[i,j] = ω
            α_store[i,j] = α
            β_store[i,j] = β
            γ_store[i,j] = γ
            ν̃_store[i,j] = ν̃
            loss_vals[i,j] = error_value
            
            α+=α_increment
            j+=1
        end
        ω+=ω_increment
        i+=1
    end
    return ω_, α_, β_, γ_, ν̃_, e, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals
end

sir_model (generic function with 1 method)

In [40]:
ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals = sir_model(infected, recovered, deceased, P, ᾱ, ρ)

  0.000464 seconds (21 allocations: 180.109 KiB)
  0.000426 seconds (21 allocations: 180.109 KiB)
  0.000428 seconds (21 allocations: 180.109 KiB)
  0.000397 seconds (21 allocations: 180.109 KiB)
  0.000414 seconds (21 allocations: 180.109 KiB)
  0.000410 seconds (21 allocations: 180.109 KiB)
  0.000432 seconds (21 allocations: 180.109 KiB)
  0.000409 seconds (21 allocations: 180.109 KiB)
  0.000406 seconds (21 allocations: 180.109 KiB)
  0.000409 seconds (21 allocations: 180.109 KiB)
  0.000414 seconds (21 allocations: 180.109 KiB)
  0.000423 seconds (21 allocations: 180.109 KiB)
  0.000881 seconds (21 allocations: 180.109 KiB)
  0.000407 seconds (21 allocations: 180.109 KiB)
  0.000776 seconds (21 allocations: 180.109 KiB)
  0.000416 seconds (21 allocations: 180.109 KiB)
  0.000816 seconds (21 allocations: 180.109 KiB)
  0.000413 seconds (21 allocations: 180.109 KiB)
  0.000436 seconds (21 allocations: 180.109 KiB)
  0.000413 seconds (21 allocations: 180.109 KiB)
  0.000416 seconds (

(0.05, 1, 2.0943379323607997e-21, 9.367652858899643e-22, 2.037669255493483e-23, 2.119559270666328e13, [0.05 0.05 … 0.05 0.05; 0.1 0.1 … 0.1 0.1; … ; 0.9500000000000003 0.9500000000000003 … 0.9500000000000003 0.9500000000000003; 0.0 0.0 … 0.0 0.0], [1.0 21.0 … 81.0 101.0; 1.0 21.0 … 81.0 101.0; … ; 1.0 21.0 … 81.0 101.0; 0.0 0.0 … 0.0 0.0], [2.0943379323607997e-21 0.0 … 0.0 0.0; 2.0841158638166904e-21 0.0 … 0.0 0.0; … ; 2.0775170868386344e-21 2.077400210890207e-21 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [9.367652858899643e-22 0.0 … 0.0 0.0; 9.365964282810024e-22 0.0 … 0.0 0.0; … ; 9.364889064640143e-22 9.368104601383983e-22 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [2.037669255493483e-23 0.0 … 0.0 0.0; 2.020783494597333e-23 0.0 … 0.0 0.0; … ; 2.0100313128984734e-23 2.0105384658361594e-23 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [2.119559270666328e13 0.0 … 0.0 0.0; 2.119559270666328e13 0.0 … 0.0 0.0; … ; 2.119559270666328e13 2.119559270666328e13 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [41]:
function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end

meshgrid (generic function with 1 method)

In [42]:
function plot_loss_curve(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)
    # ω_s,α_s = meshgrid(vec(ω_store), vec(α_store))
    surface(reshape(vec(ω_store),(num_ω,num_α)), reshape(vec(α_store),(num_ω,num_α)), reshape(vec(loss_vals),(num_ω,num_α)))
    # plot(x, y, z, zcolor = reverse(z), m = (10, 0.8, :blues, Plots.stroke(0)), leg = false, cbar = true, w = 5)
    # scatter!(ω_, α_, least_loss, w = 10)
end

plot_loss_curve (generic function with 1 method)

In [43]:
function plot_loss_tri(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)
    # ω_s,α_s = meshgrid(vec(ω_store), vec(α_store))
    PyPlot.plot_trisurf(vec(ω_store), vec(α_store), vec(loss_vals))
    # plot(x, y, z, zcolor = reverse(z), m = (10, 0.8, :blues, Plots.stroke(0)), leg = false, cbar = true, w = 5)
    # scatter!(ω_, α_, least_loss, w = 10)
end

plot_loss_tri (generic function with 1 method)

In [44]:
println(typeof(vec(ω_store)),size(vec(α_store)),size(vec(loss_vals)))

Vector{Float64}(120,)(120,)


In [46]:
# pyplot()
plot_loss_curve(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/manuvancha/.julia/packages/PyCall/3fwVL/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("'Text' object has no property 'math_fontfamily'")
  File "/Users/manuvancha/.julia/conda/3/lib/python3.9/site-packages/matplotlib/colorbar.py", line 761, in set_label
    self._set_label()
  File "/Users/manuvancha/.julia/conda/3/lib/python3.9/site-packages/matplotlib/colorbar.py", line 731, in _set_label
    self.ax.set_ylabel(self._label, **self._labelkw)
  File "/Users/manuvancha/.julia/conda/3/lib/python3.9/site-packages/matplotlib/axes/_axes.py", line 300, in set_ylabel
    return self.yaxis.set_label_text(ylabel, fontdict, **kwargs)
  File "/Users/manuvancha/.julia/conda/3/lib/python3.9/site-packages/matplotlib/axis.py", line 1566, in set_label_text
    self.label.update(kwargs)
  File "/Users/manuvancha/.julia/conda/3/lib/python3.9/site-packages

In [20]:
plot_loss_tri(ω_, α_, β_, γ_, ν̃_, least_loss, ω_store, α_store, β_store, γ_store, ν̃_store, loss_vals)

LoadError: UndefVarError: PyPlot not defined

In [21]:
pyplot()
Plots.PyPlotBackend()

┌ Warning: You are using Matplotlib 3.3.4, which is no longer
│ officialy supported by the Plots community. To ensure smooth Plots.jl
│ integration update your Matplotlib library to a version >= 3.4.0
│ 
│ If you have used Conda.jl to install PyPlot (default installation),
│ upgrade your matplotlib via Conda.jl and rebuild the PyPlot.
│ 
│ If you are not sure, here are the default instructions:
│ 
│ In Julia REPL:
│ ```
│ import Pkg;
│ Pkg.add("Conda")
│ import Conda
│ Conda.update()
│ Pkg.build("PyPlot")
│ ```
│ 
└ @ Plots /Users/manuvancha/.julia/packages/Plots/1WwM3/src/backends/pyplot.jl:29


Plots.PyPlotBackend()

In [22]:
function store_data(infected_df,recovered_df,deceased_df)
    infected_cases_data_path = "data/preprocessed/countries/time_series_covid19_confirmed_global.csv"
    recovered_cases_data_path = "data/preprocessed/countries/time_series_covid19_recovered_global.csv"
    deceased_cases_data_path = "data/preprocessed/countries/time_series_covid19_deaths_global.csv"

    CSV.write(infected_cases_data_path, infected_df)
#     infected_US = infected_total[infected_total[!, :"Country/Region"] .== "US", :]
#     infected = select(infected_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

    CSV.write(recovered_cases_data_path, recovered_df)
#     recovered_US = recovered_total[recovered_total[!, :"Country/Region"] .== "US", :]
#     recovered = select(recovered_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

    CSV.write(deceased_cases_data_path, deceased_df)
#     deceased_US = deceased_total[deceased_total[!, :"Country/Region"] .== "US", :]
#     deceased = select(deceased_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"]))#remove columns that are not required


end

store_data (generic function with 1 method)

In [23]:
#compute S̃, Ĩ and R̃ of all time t.


# if !isfile("storage/preprocessed_data.csv")
#     for t = 1:length(infected_total)
#         S̃[t] = [ω / α] * P - Ĩ[t] - R̃[t] - D̃[t]
#         Ĩ[t + 1] = Ĩ[t] + β * [[S̃[t] * Ĩ[t]] / [S̃[t] + Ĩ[t]]] - γ * Ĩ[t] - ν * Ĩ[t]
#         R̃[t + 1] = R̃[t] + γ * Ĩ[t]
#         D[t + 1] = D[t] + α * ν * Ĩ[t]
#     store_data(infected_df,recovered_df,deceased_df)
# end

# Store to avoid repeated computing.